In [1]:
import torch

In [2]:
from torch import nn

## transfer from Pytorch to Pytorch lightning

In [9]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        
        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = torch.nn.Linear(28 * 28, 128)
        self.layer_2 = torch.nn.Linear(128, 256)
        self.layer_3 = torch.nn.Linear(256, 10)
    
    def forward(self, x):
        batch_size, channels, width, height = x.size()
        
        # (b, 1, 28, 28) -> (b, 1 * 28 * 28)
        x = x.view(batch_size, -1)
        
        # layer 1
        x = self.layer_1(x)
        x = torch.relu(x)
        
        # layer 2
        x = self.layer_2(x)
        x = torch.relu(x)
        
        # layer 3
        x = self.layer_3(x)
        
        # probability distribution over labels
        x = torch.log_softmax(x, dim=1)
        
        return x

In [4]:
# to convert the pytorch module into a pytorch lightning module
import pytorch_lightning as pl

In [29]:
class LightningMNISTClassifier(pl.LightningModule):
    def __init__(self):
        super(LightningMNISTClassifier, self).__init__()
        
        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = torch.nn.Linear(28 * 28, 128)
        self.layer_2 = torch.nn.Linear(128, 256)
        self.layer_3 = torch.nn.Linear(256, 10)
    
    def forward(self, x):
        batch_size, channels, width, height = x.size()
        
        # (b, 1, 28, 28) -> (b, 1 * 28 * 28)
        x = x.view(batch_size, -1)
        
        # layer 1
        x = self.layer_1(x)
        x = torch.relu(x)
        
        # layer 2
        x = self.layer_2(x)
        x = torch.relu(x)
        
        # layer 3
        x = self.layer_3(x)
        
        # probability distribution over labels
        x = torch.log_softmax(x, dim=1)
        
        return x

In [14]:
# a Lightning module can be exactly used as a Pytorch module
pytorch_model = MNISTClassifier()
lightning_model = LightningMNISTClassifier()

In [15]:
x = torch.Tensor(32, 1, 28, 28)

In [16]:
pt_out = pytorch_model(x)
pl_out = lightning_model(x)

## Data preparation

1. Download images
2. Image transforms (highly subjective)
3. Generate training, validation and test dataset splits
4. Wrap each dataset split in a DataLoader

In [22]:
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import os
from torchvision import datasets, transforms

In [25]:
class MNISTDataModule(pl.LightningDataModule):
    def prepare_data(self):
        """handles downloads, when you use multiple GPUs,
           you don't download multiple datasets or
           apply double manipulations to the data
        """
        MNIST(os.getcwd(), train=True, download=True)
        MNIST(os.getcwd(), train=False, download=True)
    
    def train_dataloader(self):
        transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.1307,),(0.3081,))])
        mnist_train = MNIST(os.getcwd(), train=True, download=False,
                          transform=transform)
        self.mnist_train, self.mnist_val = random_split(mnist_train, [55000, 5000])
        
        mnist_train = DataLoader(mnist_train, batch_size=64)
        return mnist_train
    
    def val_dataloader(self):
        mnist_val = DataLoader(self.mnist_val, batch_size=64)
        return mnist_val
    
    def test_dataloader(self):
        transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normlaize((0.1307,),(0.3081,))])
        mnist_test = MNIST(os.getcwd(), train=False, download=False,
                         transform=transform)
        mnist_test = DataLoader(mnist_test, batch_size=64)
        return mnist_test

## Optimiser

In [30]:
class LightningMNISTClassifier(pl.LightningModule):
    def configure_optimizers(self):
        # pass in self.parameters() because the LightningModule IS the model
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

## Loss

In [31]:
from torch.nn import functional as F

In [32]:
class LightningMNISTClassifier(pl.LightningModule):
    def __init__(self):
        super(LightningMNISTClassifier, self).__init__()
        
        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = torch.nn.Linear(28 * 28, 128)
        self.layer_2 = torch.nn.Linear(128, 256)
        self.layer_3 = torch.nn.Linear(256, 10)
    
    def forward(self, x):
        batch_size, channels, width, height = x.size()
        
        # (b, 1, 28, 28) -> (b, 1 * 28 * 28)
        x = x.view(batch_size, -1)
        
        # layer 1
        x = self.layer_1(x)
        x = torch.relu(x)
        
        # layer 2
        x = self.layer_2(x)
        x = torch.relu(x)
        
        # layer 3
        x = self.layer_3(x)
        
        # probability distribution over labels
        x = torch.log_softmax(x, dim=1)
        
        return x
    
    def cross_entropy_loss(self, logits, labels):
        return F.nll_loss(logits, labels)

## Training and validation loop

1. Iterate for many epochs (an epoch is a full pass through the dataset D)
2. Each epoch iterates the dataset in small chunks called batches b
3. Perform a forward pass, compute the loss
4. Perform a backward pass to calculate all the gradients for each weight
5. Apply the gradients to each weight

In [33]:
class LightningMNISTClassifier(pl.LightningModule):
    def __init__(self):
        super(LightningMNISTClassifier, self).__init__()
        
        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = torch.nn.Linear(28 * 28, 128)
        self.layer_2 = torch.nn.Linear(128, 256)
        self.layer_3 = torch.nn.Linear(256, 10)
    
    def forward(self, x):
        batch_size, channels, width, height = x.size()
        
        # (b, 1, 28, 28) -> (b, 1 * 28 * 28)
        x = x.view(batch_size, -1)
        
        # layer 1
        x = self.layer_1(x)
        x = torch.relu(x)
        
        # layer 2
        x = self.layer_2(x)
        x = torch.relu(x)
        
        # layer 3
        x = self.layer_3(x)
        
        # probability distribution over labels
        x = torch.log_softmax(x, dim=1)
        
        return x
    
    def cross_entropy_loss(self, logits, labels):
        return F.nll_loss(logits, labels)
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('val_loss', loss)

## Trainer

In [34]:
# train loop + val loop + test loop
trainer = pl.Trainer()
trainer.fit(LightningMNISTClassifier())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


ValueError: An invalid dataloader was passed to `Trainer.fit(train_dataloaders=...)`. Either pass the dataloader to the `.fit()` method OR implement `def train_dataloader(self):` in your LightningModule/LightningDataModule.